<a href="https://colab.research.google.com/github/evillag/uncertainty_gan/blob/main/CERN_UE_Output_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

In [ ]:
!git clone https://gitlab.com/lambda-hse/lhcb-rich-gan-uncertainty.git
!mv lhcb-rich-gan-uncertainty/experiments .
!mv lhcb-rich-gan-uncertainty/src .
!rm -r lhcb-rich-gan-uncertainty/
!rm -r sample_data/
!pip install tensorflow-addons

Cloning into 'lhcb-rich-gan-uncertainty'...
remote: Enumerating objects: 210, done.
remote: Total 210 (delta 0), reused 0 (delta 0), pack-reused 210 (from 1)
Receiving objects: 100% (210/210), 2.94 MiB | 25.96 MiB/s, done.
Resolving deltas: 100% (94/94), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 4.3 MB/s eta 0:00:00


In [ ]:
from experiments.efficiency.uncertainty_model_train import train_model
from experiments.efficiency.uncertainty_models import uncertainty_mlp
from experiments.efficiency.uncertainty_utils import (
    efficiency_bands_with_uncertainty, efficiency_momentum_with_uncertainty)
from experiments.efficiency.utils import (
    efficiency_bands, efficiency_momentum, ensemble_and_ref_model_inference,
    ensemble_and_ref_model_inference_on_bands, tf_to_numpy_dataset,
    threshold_selection)
from src.cramer_gan_trainer import CramerGANTrainer
from src.dataset import CramerGANDataset
from src.datasets.utils_rich import (get_merged_typed_dataset,
                                     parse_dataset_np, parse_example)
from src.models.gans.discriminators.fcn_disc import RICHDiscriminator
from src.models.gans.generators.fcn_gen import RichMCDropFunc, VirtualEnsembleModel

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
# Dataset download and extraction
!unzip -qq drive/MyDrive/cern/data/rich.zip

In [ ]:
# Model checkpoint download and extraction
!unzip -qq drive/MyDrive/cern/data/checkpoints.zip

In [ ]:
# (New) model checkpoint download and extraction
!unzip -qq drive/MyDrive/cern/data/new_checkpoints.zip

In [ ]:
PARTICLES = ['kaon'] # ["pion", 'kaon', "muon", "proton"]
DROPOUTS = [0.25, 0.3, 0.35, 0.4]
ENSEMBLES = [16, 32, 64, 128, 256]
NUM_REPS = 10
SUB_SAMPLE_SIZE = .3
THRESHOLD = 1.0

DATA_DIR = 'rich'
CHECKPOINT_BASE = 'checkpoints'
CKPT_NUMBER = 'ckpt-21'


def get_checkpoint_name(particle):
    return f'bernoulli_structured_dropout_line_test_cramer_weighted_{particle}'

In [ ]:
def _split_by_line(df, slope=1, intercept=0):
    top_half = df[df['Brunel_ETA'] > df['Brunel_P'] * slope + intercept]
    bottom_half = df[df['Brunel_ETA'] <= df['Brunel_P'] * slope + intercept]

    top_half = top_half.reset_index(drop=True)
    bottom_half = bottom_half.reset_index(drop=True)

    return top_half, bottom_half


def split_by_line(df_train, df_test):
    return _split_by_line(df_train)[0], _split_by_line(df_test)[1]


def load_particle_datasets(particle, data_dir=DATA_DIR):
    """ The returned dictionary has this format:
        {
          "<particle_name>": {
            'data_train': data_train,
            'data_val': data_val,
            'scaler': scaler,
            'feats_train': feats_train,
            'targets_train': targets_train,
            'feats_val': feats_val,
            'targets_val': targets_val
          }
        }
    """
    data_train, data_val, scaler = get_merged_typed_dataset(data_dir, particle, dtype=np.float32, log=True,
                                                            sample_fn=split_by_line)
    feats_train, targets_train, _ = parse_dataset_np(data_train)
    feats_val, targets_val, _ = parse_dataset_np(data_val)

    print(f'feats_train shape\t{feats_train.shape}\n'
          f'targets_train shape\t{targets_train.shape}\n'
          f'feats_val shape  \t{feats_val.shape}\n'
          f'targets_val shape\t{targets_val.shape}\n')

    return {
        'data_train': data_train,
        'data_val': data_val,
        'scaler': scaler,
        'feats_train': feats_train,
        'targets_train': targets_train,
        'feats_val': feats_val,
        'targets_val': targets_val
    }

datasets = {particle: load_particle_datasets(particle) for particle in PARTICLES}

Reading and concatenating datasets:
	rich/kaon2_+_up_2016_.csv
	rich/kaon_+_up_2016_.csv
	rich/kaon2_-_down_2016_.csv
	rich/kaon_+_down_2016_.csv
	rich/kaon2_+_down_2016_.csv
	rich/kaon2_-_up_2016_.csv
	rich/kaon_-_down_2016_.csv
	rich/kaon_-_up_2016_.csv
splitting to train/val/test
fitting the scaler
scaler train sample size: 2000000
scaler n_quantiles: 100000, time = 2.261545419692993
scaling train set
scaling test set
converting dtype to <class 'numpy.float32'>
feats_train shape	(935276, 3)
targets_train shape	(935276, 5)
feats_val shape  	(532300, 3)
targets_val shape	(532300, 5)



In [ ]:
class MonteCarloDroupoutModel:
    def __init__(self, particle, dropout_rate,
                 log_dir='log_dir_tmp',
                 checkpoint_dir=CHECKPOINT_BASE,
                 debug=False):

        self.particle = particle
        self.dropout_rate = dropout_rate
        self.log_dir = log_dir
        self.checkpoint_dir = checkpoint_dir

        print(f'Generating model for {particle} with a dropout rate of {dropout_rate}')

        self._gen_config = {
            'drop_rate': dropout_rate,
            'dropout_type': 'bernoulli',
        }

        self._generator = RichMCDropFunc(**self._gen_config)
        self._generator.build((None, 3))
        self._discriminator = RICHDiscriminator()

        if debug:
            print("\nGenerator:\n")
            print(self._generator.summary(line_length=96))
            print("\nDiscriminator:\n")
            print(self._discriminator.summary())
            print(f"\nCheckpoint path: {self.checkpoint_dir}\n")

        # Model was trained with tensorflow 2.10.1, use the legacy optimizer
        self._generator_optimizer = tf.keras.optimizers.legacy.RMSprop(2e-4)
        self._discriminator_optimizer = tf.keras.optimizers.legacy.RMSprop(2e-4)

        self._trainer_config = {
            'generator': self._generator,
            'discriminator': self._discriminator,
            'generator_optimizer': self._generator_optimizer,
            'discriminator_optimizer': self._discriminator_optimizer,
            'checkpoint_dir': self.checkpoint_dir,
            'log_dir': log_dir
        }

        trainer = CramerGANTrainer(**self._trainer_config)
        # Restore pretrained model
        trainer.restore_last()

    def str(self):
        return f"{self.particle}_{self.dropout_rate}"

    def get_generator(self) -> VirtualEnsembleModel:
        return self._generator

In [ ]:
mc_model = MonteCarloDroupoutModel('kaon', 0.1, debug=True)
gen1 = mc_model.get_generator()
gen1.single_model_inference_mode()
gen1.summary()

Generating model for kaon with a dropout rate of 0.1
Layer 0
Layer 1
Layer 2
Layer 3
Layer 4

Generator:

Model: "virtual_ensemble_model"
________________________________________________________________________________________________
 Layer (type)                              Output Shape                          Param #        
 Inputs (InputLayer)                       [(None, 3)]                           0              
                                                                                                
 NoiseInjection (NoiseInjection)           (None, 67)                            0              
                                                                                                
 Layer_0/Dense (Dense)                     (None, 128)                           8704           
                                                                                                
 Layer_0/LeakyReLU (LeakyReLU)             (None, 128)                           0    

In [ ]:
def create_generator(dropout_rate, dropout_type='bernoulli'):
    gen_config = {
        'drop_rate': dropout_rate,
        'dropout_type': dropout_type,
    }
    generator = RichMCDropFunc(**gen_config)
    generator.build((None, 3))
    return generator

In [ ]:
# Solution to read embeddings of any layer:

# 1. Create a new model with the same architecture
new_gen = create_generator(0.1)
new_gen.set_weights(gen1.get_weights())

# 2. Create a new model that outputs from the layer of interest
input_layer = new_gen.input
output_layer = new_gen.layers[14].output  # Index is 14 because layer indexing starts from 0

# 3. Create a `new_model` without optimizations
new_model = Model(input_layer, [output_layer, new_gen.output])

# 4. Use `new_model` to predict on any input vector and get the embeddings
input_data = np.random.rand(1, 3)
embedding, prediction = new_model.predict(input_data)

print('Layer 4 embedding:', embedding)
print('Final prediction:', prediction)

Layer 0
Layer 1
Layer 2
Layer 3
Layer 4
1/1 [==============================] - 0s 252ms/step
Layer 4 embedding: [[ 0.17052293 -0.2815201  -0.2520858  -0.15028125  0.22142968 -0.27584785
  -0.5023401  -0.46137294 -0.3919465  -0.16284706 -0.51239127  0.12214717
  -0.04213858  0.34308702 -0.04604173 -0.2288125  -0.29522505  0.15411066
   0.51749533 -0.23563501  0.1944228   0.38582638  0.05307005 -0.28394228
  -0.04318975 -0.0614185  -0.15341006 -0.26046407 -0.36376604 -0.14553931
  -0.2214281   0.02179953 -0.19123876  0.03446744 -0.15266131 -0.00848499
  -0.23002313 -0.23316294 -0.05080584  0.52292854 -0.00468181 -0.46704197
   0.11049912 -0.3345214  -0.16706187  0.66778946 -0.11170307  0.10450091
  -0.07133051 -0.7317531  -0.32408231  0.36991304 -0.17509575 -0.02209144
  -0.10620698  0.29626992 -0.13980411  0.41269177 -0.10820699  0.26206267
   0.01205708 -0.25190696 -0.48604205  0.0670359   0.71062285 -0.07978024
  -0.20050943 -0.5262162  -0.15464188 -0.12407905 -0.43808293  0.24287237


In [ ]:
kaon_train_feats = datasets['kaon']['feats_train']
kaon_test_feats = datasets['kaon']['feats_val']
kaon_test_targets = datasets['kaon']['targets_val']

print('kaon train features shape:', kaon_train_feats.shape)
print('kaon test features shape:', kaon_test_feats.shape)
print('kaon test targets shape:', kaon_test_targets.shape)

kaon train features shape: (935276, 3)
kaon test features shape: (532300, 3)
kaon test targets shape: (532300, 5)


In [ ]:
kaon_train_embeddings = new_model.predict(kaon_train_feats)[0]
kaon_test_embeddings, kaon_test_predictions = new_model.predict(kaon_test_feats)

16635/16635 [==============================] - 41s 2ms/step


In [ ]:
dir = 'drive/MyDrive/Colab Notebooks/outputs/'

np.save(dir + 'kaon_train_embeddings.npy', kaon_train_embeddings)
np.save(dir + 'kaon_test_targets.npy', kaon_test_targets)
np.save(dir + 'kaon_test_embeddings.npy', kaon_test_embeddings)
np.save(dir + 'kaon_test_predictions.npy', kaon_test_predictions)